In [77]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from visualize import visualize_rooms
from preprocessing import preprocess_df, merge_data
from sklearn.ensemble import RandomForestRegressor
from model import train_model
from autots import AutoTS
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from autots.evaluator.benchmark import Benchmark
bench = Benchmark()
bench.run(n_jobs="auto", times=3)
bench.results

Beginning AverageValueNaive
Beginning SectionalMotif
Beginning NVAR
Beginning Datepart RandomForest
Beginning Datepart SVM
Beginning Theta
Beginning ARIMA
Beginning Multivariate KNN
Beginning MLP
Beginning KerasRNN
tensorflow failed with: ModuleNotFoundError("No module named 'keras'")
Beginning KerasCNN
tensorflow CNN failed with: ModuleNotFoundError("No module named 'keras'")
Beginning GluonTS
gluonts failed with: ImportError('GluonTS installation not found or installed version is incompatible with AutoTS.')
Beginning Prophet
prophet failed with: ModuleNotFoundError("No module named 'prophet'")
Beginning AverageValueNaive
Beginning SectionalMotif
Beginning NVAR
Beginning Datepart RandomForest
Beginning Datepart SVM
Beginning Theta
Beginning ARIMA
Beginning Multivariate KNN
Beginning MLP
Beginning KerasRNN
tensorflow failed with: ModuleNotFoundError("No module named 'keras'")
Beginning KerasCNN
tensorflow CNN failed with: ModuleNotFoundError("No module named 'keras'")
Beginning GluonTS

{'version': '0.4.0',
 'platform': 'Windows-10-10.0.22000-SP0',
 'node': 'DESKTOP-B65106C',
 'python_version': '3.9.12',
 'n_jobs': 'auto',
 'times': 3,
 'avg_naive_runtime': 0.9673456000000442,
 'sect_motif_runtime': 5.643858600000082,
 'nvar_runtime': 7.42452579999978,
 'datepart_trees_runtime': 6.63874289999967,
 'datepart_svm_runtime': 7.415726366666907,
 'multivariate_knn_runtime': 10.48839503333329,
 'theta_runtime': 13.621469433333308,
 'arima_runtime': 12.857076733333392,
 'sklearn_mlp_runtime': 1.5788178333332326,
 'total_runtime': 66.6359582999997,
 'tensorflow_rnn_runtime': 0.0,
 'tensorflow_cnn_runtime': 0.0,
 'gluonts_runtime': 0.0,
 'prophet_runtime': 0.0}

In [78]:
# load dataset and preprocess
df = pd.read_csv("data/train/data.csv", sep=";")
# merge dataset with weather data
df = merge_data(df)
# discretize, impute etc.
df = preprocess_df(df)

(140253, 6) (848411, 20)


In [ ]:
# AutoRegressor

In [ ]:
df_ts = df.copy()
df_ts = df_ts.reset_index().rename(columns={'index':'timestamp'})
print(df_ts.head(2))
metric_weighting = {
    'smape_weighting': 5,
    'mae_weighting': 2,
    'rmse_weighting': 2,
    'made_weighting': 0.5,
    'mage_weighting': 1,
    'mle_weighting': 0,
    'imle_weighting': 0,
    'spl_weighting': 3,
    'containment_weighting': 0,
    'contour_weighting': 1,
    'runtime_weighting': 0.05,
}
# Evaluate different simple time series models
model = AutoTS(
    forecast_length=96,
    frequency='infer',
    model_list="multivariate",
    transformer_list="all",
    ensemble='simple',
    max_generations=20,
    num_validations=2,
    metric_weighting=metric_weighting,
    n_jobs='auto',
    holiday_country="DE",
    validation_method="backwards"
)

model.fit(df_ts, date_col='timestamp', value_col='RoomTemperature', id_col="Room")

In [69]:
#roomfour = df[df["Room"]=="Raum 004"]
#model=ETSModel(roomfour)
#fit=model.fit(maxiter=1000)
#X=df_ts[:,:-1]
#y=df_ts[:,-1]
#print(df.index)
#df=df.index

df_ts = df.copy()
df_ts = df_ts.reset_index().rename(columns={'index':'timestamp'})
X,y=df_ts.drop(['RoomTemperature'], axis=1), df_ts['RoomTemperature']
X['timestamp']=[(int(elem.timestamp()))%7 for elem in X['timestamp']]
model=MLPRegressor(random_state=1, max_iter=300,verbose=True).fit(X, y)

Iteration 1, loss = 6.12824612
Iteration 2, loss = 0.29221966
Iteration 3, loss = 0.22378249
Iteration 4, loss = 0.19912970
Iteration 5, loss = 0.18698228
Iteration 6, loss = 0.17930021
Iteration 7, loss = 0.17403015
Iteration 8, loss = 0.17046356
Iteration 9, loss = 0.16748388
Iteration 10, loss = 0.16504452
Iteration 11, loss = 0.16319854
Iteration 12, loss = 0.16072004
Iteration 13, loss = 0.15939499
Iteration 14, loss = 0.15775809
Iteration 15, loss = 0.15660681
Iteration 16, loss = 0.15510819
Iteration 17, loss = 0.15401781
Iteration 18, loss = 0.15316896
Iteration 19, loss = 0.15219019
Iteration 20, loss = 0.15147132
Iteration 21, loss = 0.15073778
Iteration 22, loss = 0.14995406
Iteration 23, loss = 0.14918801
Iteration 24, loss = 0.14902369
Iteration 25, loss = 0.14813625
Iteration 26, loss = 0.14774502
Iteration 27, loss = 0.14762732
Iteration 28, loss = 0.14694145
Iteration 29, loss = 0.14662518
Iteration 30, loss = 0.14629679
Iteration 31, loss = 0.14596139
Iteration 32, los

In [75]:

"""df_test = pd.read_csv("data/test/test_feb2022.csv", sep=",")
df_test = merge_data(df_test)
# discretize, impute etc.
df_test = preprocess_df(df_test)
df_test_ts = df_test.copy()
df_test_ts = df_test_ts.reset_index().rename(columns={'index':'timestamp'})
X_test,y_test=df_test_ts.drop(['RoomTemperature'], axis=1), df_test_ts['RoomTemperature']
X_test['timestamp']=[(int(elem.timestamp()))%7 for elem in X_test['timestamp']]
"""
rms = mean_squared_error(y, model.predict(X), squared=False)
print(rms)

0.532465699366784


In [4]:
# train your model then:
# set n=1 if you only want your best model
model.export_template("my_models.csv", models='best', n=15, max_per_model_class=3)

# later on a new session
# you can set `max_generations=0` in model, and then it will only attempt the imported models
# model = model.import_template("my_models.csv", method='only')